<a href="https://colab.research.google.com/github/asadstat/Data_analysis/blob/master/bert_balance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/sentiment.csv',usecols=['domain','title','content','Sentiment'])
df= df[df['Sentiment'] != "Nutral"]
df.head()

In [ ]:
df.rename(columns = {'Sentiment':'Category', 'content':'Message'}, inplace = True)
df.head()

In [ ]:
df['Sentiment']=df['Category'].apply(lambda x: 1 if x=='Positive' else 0)
df.head()

In [ ]:
df['Category'].value_counts()

Positive    22367
Negative     2536
Name: Category, dtype: int64

In [ ]:
df_positive = df[df['Category']=='Positive']

In [ ]:
df_negative = df[df['Category']=='Negative']

In [ ]:
df_positive_downsampled = df_positive.sample(df_negative.shape[0])

In [ ]:
df_balanced = pd.concat([df_positive_downsampled, df_negative])

In [ ]:
df_balanced['Category'].value_counts()

Positive    2536
Negative    2536
Name: Category, dtype: int64

In [ ]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='Yes' else 0)
df_balanced.head()

,domain,title,Message,Category,Comment,spam
22963,qz,Coronavirus has LVMH and Foxconn making medica...,To show they’re not just concerned with profit...,Positive,1,0
7901,cnbc,"Rio Tinto posts best earnings since 2011, read...",Feb 26 (Reuters) - Anglo-Australian miner Rio...,Positive,1,0
5151,nytimes,‘Thermometer Guns’ on Coronavirus Front Lines ...,It has become an iconic image of the coronavir...,Positive,1,0
4676,express,UK coronavirus: Can I catch coronavirus from m...,"UK coronavirus cases have progressed quickly,...",Positive,1,0
19920,cnbc,"Disney shutting theme parks in California, Flo...","LOS ANGELES, March 12 (Reuters) - Walt Disney...",Positive,1,0


In [ ]:
df_balanced.sample(5)

,domain,title,Message,Category,Comment,spam
7052,fool,Why Facebook Stock Fell Monday,Shares of Facebook (NASDAQ:FB) took a hit on M...,Negative,0,0
3123,cnbc,Millions of chickens at risk amid China lockdo...,"Following its pork crisis, China's poultry far...",Negative,0,0
2521,nytimes,It’s Almost Fashion Month! Here’s What to Know,It is often said that when President Trump eve...,Positive,1,0
11944,theatlantic,Who Should Avoid Crowds Because of the Coronav...,Editor's Note: Information about the novel cor...,Positive,1,0
12970,marketwatch,All sporting events in Italy will be played wi...,MILAN (AP) — All sporting events in Italy will...,Negative,0,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])
X_train.head(4)

In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

RuntimeError: ignored

ImportError: ignored

In [ ]:
#importing neccessary modules
import os
import tensorflow
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4")
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_bert = model.fit(X_train, y_train, epochs=10, batch_size = 32)

In [ ]:
history_bert =model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=10, 
          batch_size=32, 
          verbose=1)

In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history_bert):
    acc = history_bert.history['accuracy']
    loss = history_bert.history['loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
print(history_bert.history.keys())

In [ ]:
plot_history(history_bert)

In [ ]:
pred_bert = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,pred_bert))

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# Calculate the accuracy score
accuracy_bert = metrics.accuracy_score(y_test,pred_bert)
print(accuracy_bert)
Conf_metrics_bert = metrics.confusion_matrix(y_test,pred_bert, labels=['1', '0'])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
labels = ['1', '0']
print(classification_report(y_test, pred_bert))
pd.DataFrame(confusion_matrix(y_test, pred_bert), index=labels, columns=labels)

In [ ]:
conf_bert = metrics.confusion_matrix(y_test,pred_bert)
conf_bert

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

np.random.seed(42)  # for reproducibility
sns.set(rc={"figure.figsize": (4, 4)})
sns.set_style("ticks")

In [ ]:
classes = [0, 1]
# plot confusion matrix
plt.imshow(conf_bert, interpolation='nearest', cmap=plt.cm.Greens)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

fmt = 'd'
thresh = conf_bert.max() / 2.
for i, j in itertools.product(range(conf_bert.shape[0]), range(conf_bert.shape[1])):
    plt.text(j, i, format(conf_bert[i, j], fmt),
             horizontalalignment="center",
             color="white" if conf_bert[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
probs = model.predict(X_test)
preds=probs 

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()